In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Problem Anaylsis
We want to comapre 2 documents (a resume and job description) and rank the resumes according to therir revlance, this needs a computional defintion of relevance.

### Ranking Function depends on
1- BOW representation for words (according to many courses I studied and searching through the internet), and my guess for that, is that we don't need contextual representation for words, we only need one vector for each word to be able to measure similarity between 2 documents.

2- TF-IDF,and this is for the same reason above, we don't need SBERT to represent every word, we're only looking for occurrence of keywords, we're not looking through a story.

3-Document Length
### Text Retreival Methods
1- simalirity based models (for example: vector space model or VSM)

2- probalistic models 
and others..


At our case we could use first method, in which we rank the resume based on how much it's similar to the job describtion




In [2]:
import nltk
nltk.download('popular')
from nltk.corpus import stopwords


import string

In [3]:
import re

In [139]:
resumes = pd.read_csv('/kaggle/input/summlink-resumes-matching-score/resumes.csv')
#resumes.iloc[:, 1].values
#resumes.loc[:, 'resume_text']


#### creating dictionary of keys as person name, values as his/her resume

In [5]:
resumes_dict = {}
for i in range(len(resumes)):
    resumes_dict[resumes.iloc[i, 0]] = resumes.iloc[i, 1]


In [6]:
def tokenized_list(resume_doc):
    resume_doc = str(resume_doc)
    resume_doc = resume_doc.replace('[' , ' ')
    resume_doc = resume_doc.replace(']' , ' ')
    tokens = nltk.word_tokenize(resume_doc.lower().strip())
    return tokens

In [7]:
def stemmer(tokens):
    ps = nltk.stem.PorterStemmer()
    stemmed = []
    for word in tokens:
        stemmed.append(ps.stem(word))
    return stemmed    
        

In [196]:
stop_words = set(stopwords.words('english'))
def remove_stops(doc):
    clean = []
    #doc = re.sub(r'(.)\1+', r' ', doc) #removing aaaaaaaaaa for similarity measuers
    print(doc)
    for word in doc:
        if word not in stop_words:
            clean.append(word)
    return clean        
    
    

### removing 'AAAAAA' words for computation resources  

In [197]:
def a_letters(a_word):
    i =0
    for l in a_word:
        if l == 'A':
            i+=1
        if i>= 3:
            a_word = " "
    return a_word


In [200]:
#clean_corpus  

In [ ]:
clean_corpus = []
for doc in resumes_dict.values():
    #match = re.search(r'(A)\1+', doc)
    #new_match = a_letters(match.group())
    #print(new_match)
    doc = re.sub(r'(A)\1+',lambda x: a_letters(x.group()), doc)
    tokens = tokenized_list(doc)
    doc = remove_stops(tokens)
    doc = stemmer(doc)
    doc = ' '.join(doc)
    clean_corpus.append(doc)
#clean_corpus    
    
    

In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [215]:
vecotrizer = TfidfVectorizer(sublinear_tf = True)
vecotrizer.fit(clean_corpus)
corpus_space_vector = vecotrizer.transform(clean_corpus)

#print(vecotrizer.get_feature_names())
print(corpus_space_vector.shape)

In [ ]:
df = pd.DataFrame(corpus_space_vector.toarray(), columns = vecotrizer.get_feature_names())
df.iloc[:, 2000:2100]

In [224]:
import json

In [232]:
job_description = pd.read_json('../input/summlink-resumes-matching-score/job_description_response.json')
job_description.loc[:, 'description'].values

### evaluating the necessary skills, we do this so can filter top 10 for example when evaluating addtional skills
### but for submission file we will compute for all 32 anyway

In [283]:
must_have_skills_query = " experience as a business analyst in a similar environment.\
                            experience with business process development and optimization BPM\
                            experience with stakeholder management\
                            experience within the Belgian energy sector\
                            experience with writing processes and instructions\
                            experience with SAP\
                           "
clean_must_query = []
tokens = tokenized_list(must_have_skills_query)
must_have_skills_query = remove_stops(tokens)
must_have_skills_query = stemmer(must_have_skills_query)
must_have_skills_query = ' '.join(must_have_skills_query)
clean_must_query.append(must_have_skills_query)
must_query_space_vector = vecotrizer.transform(clean_must_query)

In [296]:
from sklearn.metrics.pairwise import cosine_similarity
must_cosine_simi = cosine_similarity(corpus_space_vector, must_query_space_vector).flatten()
must_skills_rank = must_cosine_simi.argsort()[:-32:-1]
must_skills_rank

### evaluating the addtional skills for top candidates who had the necessary skills

In [297]:
add_have_skills_query = "experience in training end users.\
                      experience working on large, complex projects\
                      "

clean_add_query = []
tokens = tokenized_list(add_have_skills_query)
add_have_skills_query = remove_stops(tokens)
add_have_skills_query = stemmer(add_have_skills_query)
add_have_skills_query = ' '.join(add_have_skills_query)
clean_add_query.append(add_have_skills_query)
add_query_space_vector = vecotrizer.transform(clean_add_query)

In [298]:
add_cosine_simi = cosine_similarity(corpus_space_vector, add_query_space_vector).flatten()
add_skills_rank = add_cosine_simi.argsort()[:-32:-1]
add_skills_rank

### adding 2 scores to determine overall top 32 resumes

In [301]:
total_simi = must_cosine_simi + add_cosine_simi
onerall_rank = total_simi.argsort()[:-32:-1]
onerall_rank

In [306]:
simi_list = total_simi.tolist().sort()

### adding score colmun to original dataframe 

In [321]:
resumes_with_score = resumes.copy()
resumes_with_score['rank'] = total_simi

resumes_with_score = resumes_with_score.sort_values('rank',ascending=False)

In [338]:
resumes_with_score = resumes_with_score.drop(resumes_with_score.columns[1], axis=1)
resumes_with_score

In [340]:
resumes_with_score.to_csv('submission.csv',index = False)